In [1]:
from util import PATH_FOR_OUR_TRAINING_DATA, TARGET_SHAPE
import pandas as pd
from DS_Generator import DS_Generator
import tensorflow as tf
import datetime
import os

df = pd.read_csv(PATH_FOR_OUR_TRAINING_DATA + "/data.csv", index_col=0)

train_ds, val_ds = DS_Generator().generate(df,0.1,1,batch_size=32,one_hot_encode=True)

We threw away the datapoint with index 44763 


In [2]:
%load_ext tensorboard

In [3]:
train_ds= train_ds.take(10)

## Model

In [4]:
num_classes = len({i for i in df["species_label"]})

base_cnn = tf.keras.applications.resnet.ResNet50(
    weights=None, input_shape=TARGET_SHAPE + (1,), include_top=False
)

flatten = tf.keras.layers.Flatten()(base_cnn.output)
dense1 = tf.keras.layers.Dense(512, activation="relu")(flatten)
dense1 = tf.keras.layers.BatchNormalization()(dense1)
dense2 = tf.keras.layers.Dense(256, activation="relu")(dense1)
dense2 = tf.keras.layers.BatchNormalization()(dense2)
output = tf.keras.layers.Dense(num_classes,activation="softmax")(dense2)

model = tf.keras.Model(base_cnn.input, output, name="Resnet50Softmax")
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

## Callbacks - tensorboard + saving model

In [5]:
model_dir = "training_information/"+model.name

if model.name not in os.listdir("training_information/"):
    os.makedirs(model_dir)
    # then also create log + weights folders

if not "weights" in os.listdir(model_dir):
    os.makedirs(model_dir + "/weights")

if  not "logs" in os.listdir(model_dir):
    os.makedirs(model_dir + "/logs")

time_stamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

log_path = model_dir + "/logs/" + time_stamp
weights_path = model_dir + "/weights/" + time_stamp+ "-{epoch:04d}.ckpt"

# tensorboard
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_path, histogram_freq=1)

# saving weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=weights_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [6]:
%tensorboard --logdir Resnet50Softmax/logs

Reusing TensorBoard on port 6006 (pid 13240), started 0:08:50 ago. (Use '!kill 13240' to kill it.)

In [ ]:
model.fit(
    train_ds,
    epochs=15,
    validation_data=val_ds,
    callbacks=[cp_callback,tensorboard_callback]
)

Epoch 1/15
10/10 [==============================] - ETA: 0s - loss: 3.7437 - accuracy: 0.1281